<img src="https://assets-global.website-files.com/61ddc75d84e094baf5b1bc86/61f44e2e0cb9955d7de6cc46_modelbit-logo-with-name.svg" width="400" alt="" class="brand">

# 🏎 Quickstart

Use Modelbit for deploying ML models to production endpoints, training models on GPUs, and serving ML models in production.

## 🔬 Modelbit's Python API that automatically detects your model and its depencies

With just one line of code, you can deploy any ML model to an isolated container behind a REST API. Modelbit gives you an interface and MLOps features to manage your model in production.

<img class="feature-image" src="https://assets-global.website-files.com/61ddc75d84e094baf5b1bc86/65af0d3a1491e90d61eaf38b_Screenshot%202024-01-22%20at%204.49.49%20PM.png" alt="Sample data rows and results metadata from a Modelbit dataset" style="opacity: 1;" sizes="(max-width: 479px) 100vw, (max-width: 767px) 95vw, (max-width: 991px) 78vw, 52vw" data-w-id="49576e9b-be18-e906-bbb2-06412cb48b35" loading="lazy" srcset="https://assets-global.website-files.com/61ddc75d84e094baf5b1bc86/65af0d3a1491e90d61eaf38b_Screenshot%202024-01-22%20at%204.49.49%20PM-p-500.png 500w, https://assets-global.website-files.com/61ddc75d84e094baf5b1bc86/65af0d3a1491e90d61eaf38b_Screenshot%202024-01-22%20at%204.49.49%20PM-p-800.png 800w, https://assets-global.website-files.com/61ddc75d84e094baf5b1bc86/65af0d3a1491e90d61eaf38b_Screenshot%202024-01-22%20at%204.49.49%20PM-p-1080.png 1080w, https://assets-global.website-files.com/61ddc75d84e094baf5b1bc86/65af0d3a1491e90d61eaf38b_Screenshot%202024-01-22%20at%204.49.49%20PM.png 1573w">

## 💾 Install `Modelbit` and login


Once you have installed Modelbit and logged in to your free account, we will start by deploying a simple function that doubles numbers.

In [ ]:
pip install --upgrade modelbit

In [ ]:
import modelbit
mb = modelbit.login()

## 🚀 Deploy a model!

This step is required, but after this you can train and deploy any model you like!

In [ ]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm.fit([[1], [2], [3]], [2, 4, 6])

def example_doubler(half: int) -> int:
    if type(half) is not int:
        return None
    return round(lm.predict([[half]])[0])

mb.deploy(example_doubler)

---

# 🖼 Deploy a Segment Anything Model from Meta AI

## 👩🏽‍💻 Setup

In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

In [ ]:
from segment_anything import sam_model_registry, SamPredictor
import cv2
import urllib
import numpy as np
import matplotlib.pyplot as plt

## 📊 Building Blocks For Working With Images

In [ ]:
# Given the pixels of an image mask, return the mask's bounding box
def mask2boundingbox(mask):
    x_min = None
    x_max = None
    y_min = None
    y_max = None
    for y, row in enumerate(mask):
      for x, val in enumerate(row):
        if val:
          if x_min is None or x_min > x:
            x_min = x
          if y_min is None or y_min > y:
            y_min = y
          if x_max is None or x_max < x:
            x_max = x
          if y_max is None or y_max < y:
            y_max = y
    return x_min, y_min, x_max, y_max

# Render a mask in matplotlib
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

# Render a point as a star in matplotlib
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

# Render a box in matplotlib
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='red', facecolor=(0,0,0,0), lw=2))

# Render an image in matplotlib
def show_image(img, points = None, mask = np.ndarray([]), box = ()):
    im = plt.figure(figsize=(10,10))
    plt.imshow(img)
    if points:
        show_points(np.array([[points[0], points[1]]]), np.array([1]), plt.gca())
    if mask.any():
        show_mask(mask, plt.gca())
    if box:
        show_box(box, plt.gca())
    plt.axis('on')
    modelbit.log_image(im)
    plt.show()

## 🧠 Getting Segment Anything Running

In [ ]:
sam = sam_model_registry["default"](checkpoint="sam_vit_h_4b8939.pth")
sam.to(device="cuda")
predictor = SamPredictor(sam)

In [ ]:
def find_cat(url, x_coord, y_coord):
    url_response = urllib.request.urlopen(url)
    img = cv2.imdecode(np.array(bytearray(url_response.read()), dtype=np.uint8), -1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    predictor.set_image(img)

    masks, scores, logits = predictor.predict(
        point_coords=np.array([[x_coord, y_coord]]),
        point_labels=np.array([1]),
        multimask_output=True,
    )

    top_score = 0
    best_mask = None
    for i, score in enumerate(scores):
      if score > top_score:
        top_score = score
        best_mask = masks[i]

    bbox = mask2boundingbox(best_mask)
    show_image(img, (x_coord, y_coord), best_mask, bbox)

    return bbox

In [ ]:
bounding_box = find_cat("https://montereyzoo.org/wp-content/uploads/2017/04/big-cats-6.jpg", 225, 150)
bounding_box

## 🚀 Deploying Segment Anything to a REST API

In [ ]:
mb.deploy(find_cat,
          python_packages=[
            "git+https://github.com/facebookresearch/segment-anything.git",
            "opencv-python==4.8.0.76",
            "pycocotools==2.0.7",
            "matplotlib==3.7.1",
            "numpy==1.25.2",
            "torch==2.0.1+cu118",
            "torchvision==0.15.2+cu118",
          ],
          system_packages=["python3-opencv", "build-essential"],
          require_gpu=True
)